In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, MultiHeadAttention, Conv1D, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder

In [2]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('WNL_ESNI.csv', header=None, names=['target', *(f'C{i:02}' for i in range(5276))])
df = df.drop(0, axis=0)

In [5]:
df

,target,C00,C01,C02,C03,C04,C05,C06,C07,C08,...,C5266,C5267,C5268,C5269,C5270,C5271,C5272,C5273,C5274,C5275
1,AppleMusic,22.0,3.0,1.0,2.0,0.0,1.0,0.0,1.0,252.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AppleMusic,22.0,3.0,1.0,2.0,71.0,1.0,0.0,2.0,67.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AppleMusic,22.0,3.0,1.0,2.0,0.0,1.0,0.0,1.0,252.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AppleMusic,22.0,3.0,1.0,2.0,71.0,1.0,0.0,2.0,67.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AppleMusic,22.0,3.0,1.0,2.0,71.0,1.0,0.0,2.0,67.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3542,YouTube_PC,22.0,3.0,1.0,0.0,215.0,1.0,0.0,0.0,211.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3543,YouTube_PC,22.0,3.0,1.0,0.0,215.0,1.0,0.0,0.0,211.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3544,YouTube_PC,22.0,3.0,1.0,0.0,215.0,1.0,0.0,0.0,211.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3545,YouTube_PC,22.0,3.0,1.0,2.0,0.0,1.0,0.0,1.0,252.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.replace(r'\t', '', regex=True, inplace=True)
target = df.pop('target')

In [7]:
target

,target
1,AppleMusic
2,AppleMusic
3,AppleMusic
4,AppleMusic
5,AppleMusic
...,...
3542,YouTube_PC
3543,YouTube_PC
3544,YouTube_PC
3545,YouTube_PC


In [8]:
df

,C00,C01,C02,C03,C04,C05,C06,C07,C08,C09,...,C5266,C5267,C5268,C5269,C5270,C5271,C5272,C5273,C5274,C5275
1,22.0,3.0,1.0,2.0,0.0,1.0,0.0,1.0,252.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,22.0,3.0,1.0,2.0,71.0,1.0,0.0,2.0,67.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22.0,3.0,1.0,2.0,0.0,1.0,0.0,1.0,252.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,22.0,3.0,1.0,2.0,71.0,1.0,0.0,2.0,67.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,22.0,3.0,1.0,2.0,71.0,1.0,0.0,2.0,67.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3542,22.0,3.0,1.0,0.0,215.0,1.0,0.0,0.0,211.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3543,22.0,3.0,1.0,0.0,215.0,1.0,0.0,0.0,211.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3544,22.0,3.0,1.0,0.0,215.0,1.0,0.0,0.0,211.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3545,22.0,3.0,1.0,2.0,0.0,1.0,0.0,1.0,252.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df1=df.iloc[:, 0:2358]
df1=df1.fillna(0)
df1=df1.astype(int)
df1_normalized = df1 / 255.0
df1_normalized

,C00,C01,C02,C03,C04,C05,C06,C07,C08,C09,...,C2348,C2349,C2350,C2351,C2352,C2353,C2354,C2355,C2356,C2357
1,0.086275,0.011765,0.003922,0.007843,0.000000,0.003922,0.0,0.003922,0.988235,0.011765,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.086275,0.011765,0.003922,0.007843,0.278431,0.003922,0.0,0.007843,0.262745,0.011765,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.086275,0.011765,0.003922,0.007843,0.000000,0.003922,0.0,0.003922,0.988235,0.011765,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.086275,0.011765,0.003922,0.007843,0.278431,0.003922,0.0,0.007843,0.262745,0.011765,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.086275,0.011765,0.003922,0.007843,0.278431,0.003922,0.0,0.007843,0.262745,0.011765,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3542,0.086275,0.011765,0.003922,0.000000,0.843137,0.003922,0.0,0.000000,0.827451,0.011765,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3543,0.086275,0.011765,0.003922,0.000000,0.843137,0.003922,0.0,0.000000,0.827451,0.011765,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3544,0.086275,0.011765,0.003922,0.000000,0.843137,0.003922,0.0,0.000000,0.827451,0.011765,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3545,0.086275,0.011765,0.003922,0.007843,0.000000,0.003922,0.0,0.003922,0.988235,0.011765,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:

num_rows = 3546
num_columns = 2358

# Create an example DataFrame with random values


# Convert DataFrame to numpy array
array_df1 = df1_normalized.to_numpy()

# Reshape the numpy array to (number_of_rows, 3, 786)
# Ensure that the total number of elements matches: 3490 * 2358 = 3490 * (3 * 786)
new_shape = (num_rows, 3, 786)
x = array_df1.reshape(new_shape)



In [11]:
x.shape

(3546, 3, 786)

In [13]:
from sklearn.preprocessing import OneHotEncoder

# Convert target to a NumPy array and reshape
target = target.to_numpy().reshape(-1, 1)

# Initialize and fit the encoder
encoder = OneHotEncoder(sparse_output=False, categories='auto')
one_hot_encoded_target = encoder.fit_transform(target)

print(one_hot_encoded_target.shape)


(3546, 12)


In [14]:
label_encoder = LabelEncoder()

# Fit and transform the target values
encoded_labels = label_encoder.fit_transform(target)

# Convert the encoded labels to a pandas Series
target_series = pd.Series(encoded_labels)

# Optionally, if you need to convert it to a DataFrame
target_df = pd.DataFrame({'Encoded_Label': target_series})

print(target_series)
print(target_df)

0        0
1        0
2        0
3        0
4        0
        ..
3541    10
3542    10
3543    10
3544    10
3545    10
Length: 3546, dtype: int64
      Encoded_Label
0                 0
1                 0
2                 0
3                 0
4                 0
...             ...
3541             10
3542             10
3543             10
3544             10
3545             10

[3546 rows x 1 columns]


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
target.shape

(3546, 1)

In [16]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd

# Parameters
embedding_size = 432  # Size of the embedding
num_heads = 3  # Number of attention heads
num_blocks = 3  # Number of multi-head attention blocks
d_model = embedding_size  # Dimension of the model
kernel_number = 432  # Number of filters for 1D CNN
kernel_size = 1  # Kernel size for 1D CNN
num_classes = 12  # Number of classification labels
input_features = 786  # Number of features in each packet

# Define position encoding
def position_encoding(position, embedding_dim):
    """
    Generate position encoding using sine and cosine functions.
    """
    position = np.arange(position)[:, np.newaxis]
    div_term = np.exp(np.arange(0, embedding_dim, 2) * -(np.log(10000.0) / embedding_dim))
    pos_enc = np.zeros((position.shape[0], embedding_dim))
    pos_enc[:, 0::2] = np.sin(position * div_term)
    pos_enc[:, 1::2] = np.cos(position * div_term)
    return tf.convert_to_tensor(pos_enc, dtype=tf.float32)

class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.depth = d_model // num_heads
        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)
        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q):
        batch_size = tf.shape(q)[0]
        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)
        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)
        attention_output, _ = self.scaled_dot_product_attention(q, k, v)
        attention_output = tf.transpose(attention_output, perm=[0, 2, 1, 3])
        attention_output = tf.reshape(attention_output, (batch_size, -1, self.d_model))
        return self.dense(attention_output)

    def scaled_dot_product_attention(self, q, k, v):
        matmul_qk = tf.matmul(q, k, transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, v)
        return output, attention_weights

class FeedForwardLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, ff_dim):
        super(FeedForwardLayer, self).__init__()
        self.dense1 = tf.keras.layers.Dense(ff_dim, activation='relu')
        self.dense2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        return x

class AddNorm(tf.keras.layers.Layer):
    def __init__(self, d_model, dropout_rate):
        super(AddNorm, self).__init__()
        self.layernorm = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

    def call(self, x, sublayer_output):
        out = x + sublayer_output
        out = self.dropout(out)
        return self.layernorm(out)

class EncoderBlock(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim, dropout_rate):
        super(EncoderBlock, self).__init__()
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = FeedForwardLayer(d_model, ff_dim)
        self.add_norm1 = AddNorm(d_model, dropout_rate)
        self.add_norm2 = AddNorm(d_model, dropout_rate)

    def call(self, x):
        attn_output = self.mha(x, x, x)
        out1 = self.add_norm1(x, attn_output)
        ffn_output = self.ffn(out1)
        return self.add_norm2(out1, ffn_output)

class MATECModel(tf.keras.Model):
    def __init__(self, embedding_size, num_heads, num_blocks, ff_dim, num_classes, dropout_rate, max_seq_len):
        super(MATECModel, self).__init__()
        self.embedding_layer = tf.keras.layers.Dense(embedding_size)
        self.position_encoding = position_encoding(max_seq_len, embedding_size)
        self.encoder_blocks = [EncoderBlock(embedding_size, num_heads, ff_dim, dropout_rate) for _ in range(num_blocks)]
        self.global_avg_pool = tf.keras.layers.GlobalAveragePooling1D()
        self.final_dense = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, x):
        x = self.embedding_layer(x)
        seq_len = tf.shape(x)[1]
        x += self.position_encoding[:seq_len]
        for block in self.encoder_blocks:
            x = block(x)
        x = self.global_avg_pool(x)
        return self.final_dense(x)

# Example usage




# Parameters
num_packets = 3  # Number of packets in a flow
input_shape = (num_packets, input_features)

# Generate synthetic data

X_train, X_test, y_train, y_test = train_test_split(x, one_hot_encoded_target, test_size=0.2, random_state=42)

# Reshape data to fit the model's expected input
num_replicates = 200

# Initialize lists to store replicated test sets
X_test_replicates = []
y_test_replicates = []

for _ in range(num_replicates):
    # Append a copy of the test data to the lists
    X_test_replicates.append(X_test.copy())
    y_test_replicates.append(y_test.copy())

# Optionally convert lists to numpy arrays for easier manipulation
X_test_replicates = np.array(X_test_replicates)
y_test_replicates = np.array(y_test_replicates)

# Print shapes of the replicated test sets to verify
print(f"Shape of each X_test replicate: {X_test.shape}")
print(f"Shape of each y_test replicate: {y_test.shape}")
print(f"Total number of X_test replicates: {len(X_test_replicates)}")
print(f"Total number of y_test replicates: {len(y_test_replicates)}")



# Create and compile the model
model = MATECModel(
    embedding_size=432,
    num_heads=3,
    num_blocks=2,
    ff_dim=2048,
    num_classes=12,
    dropout_rate=0.1,
    max_seq_len=786
)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=25, batch_size=32, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')
loss, accuracy = model.evaluate(X_test, y_test)
y_pred=model.predict(X_test)
# Predict labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert y_test from one-hot encoding to class labels
# If y_test is one-hot encoded, use np.argmax to get class labels
y_test_classes = np.argmax(y_test, axis=1)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)

# Calculate error rates per class
class_totals = np.sum(conf_matrix, axis=1)  # Total instances per class
class_errors = class_totals - np.diag(conf_matrix)  # Misclassified instances per class
class_error_rates = class_errors / class_totals  # Error rate per class
class_error_rates_percentage = class_error_rates * 100  # Convert to percentage

# Create a DataFrame for class-wise error rates
# Get original class labels from the OneHotEncoder
class_labels = encoder.categories_[0]  # Assuming encoder.categories_ is an array of class labels

# Ensure that the class_labels match the order used in the confusion matrix
error_rate_df = pd.DataFrame({
    'Class': class_labels,
    'Error Rate (%)': class_error_rates_percentage
})

# Print class-wise error rate table
print("\nClass-wise Error Rate Table:")
print(error_rate_df)


Shape of each X_test replicate: (710, 3, 786)
Shape of each y_test replicate: (710, 12)
Total number of X_test replicates: 200
Total number of y_test replicates: 200
Epoch 1/25
80/80 ━━━━━━━━━━━━━━━━━━━━ 86s 979ms/step - accuracy: 0.1483 - loss: 4.1637 - val_accuracy: 0.1338 - val_loss: 2.3846
Epoch 2/25
80/80 ━━━━━━━━━━━━━━━━━━━━ 81s 960ms/step - accuracy: 0.2196 - loss: 2.2127 - val_accuracy: 0.2887 - val_loss: 1.7888
Epoch 3/25
80/80 ━━━━━━━━━━━━━━━━━━━━ 79s 931ms/step - accuracy: 0.4533 - loss: 1.5034 - val_accuracy: 0.5176 - val_loss: 2.0507
Epoch 4/25
80/80 ━━━━━━━━━━━━━━━━━━━━ 76s 950ms/step - accuracy: 0.6644 - loss: 1.0413 - val_accuracy: 0.7993 - val_loss: 0.6822
Epoch 5/25
80/80 ━━━━━━━━━━━━━━━━━━━━ 89s 1s/step - accuracy: 0.7493 - loss: 0.7475 - val_accuracy: 0.7923 - val_loss: 0.6290
Epoch 6/25
80/80 ━━━━━━━━━━━━━━━━━━━━ 135s 961ms/step - accuracy: 0.8018 - loss: 0.6115 - val_accuracy: 0.8099 - val_loss: 0.6712
Epoch 7/25
80/80 ━━━━━━━━━━━━━━━━━━━━ 78s 978ms/step - accurac